In [2]:
#from google.colab import drive
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sn
from sklearn.metrics import confusion_matrix
#import torch
#import torchvision
#import torchtext
from sklearn.preprocessing import LabelEncoder # 라벨 인코더

import os

#os.environ["CUDA_VISIBLE_DEVICES"]= ""  



In [3]:
from tensorflow.python.client import device_lib
import tensorflow as tf
print(tf.test.is_gpu_available())
print(device_lib.list_local_devices())
print()

True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1110771377565522077
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4937233203
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16404981073241555991
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]



In [4]:
import tensorflow as tf 
tf.__version__

'1.15.0'

In [5]:
import tensorflow as tf
"""
a = tf.constant(1)

b = tf.constant(2)

sess = tf.Session()

c = a + b

print('Sum = ', sess.run(c))
"""

"\na = tf.constant(1)\n\nb = tf.constant(2)\n\nsess = tf.Session()\n\nc = a + b\n\nprint('Sum = ', sess.run(c))\n"

In [6]:
from keras import backend as K
#K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


In [7]:
#데이터 위치 train,test, sample_submission
#https://www.dacon.io/competitions/official/235554/overview/

In [12]:
train=pd.read_csv('\data/train.csv')
train.head()

,layer_1,layer_2,layer_3,layer_4,0,1,2,3,4,5,...,216,217,218,219,220,221,222,223,224,225
0,10,10,10,10,0.254551,0.258823,0.254659,0.252085,0.247678,0.253614,...,0.354750,0.369223,0.388184,0.408496,0.414564,0.429403,0.419225,0.443250,0.433414,0.465502
1,10,10,10,20,0.205062,0.225544,0.217758,0.202169,0.199633,0.207380,...,0.557203,0.573656,0.587998,0.612754,0.627825,0.633393,0.637706,0.625981,0.653231,0.637853
2,10,10,10,30,0.189196,0.165869,0.177655,0.156822,0.175094,0.177755,...,0.699864,0.708688,0.721982,0.713464,0.743030,0.741709,0.747743,0.746037,0.737356,0.750391
3,10,10,10,40,0.131003,0.120076,0.138975,0.117931,0.130566,0.131262,...,0.764786,0.763788,0.770017,0.787571,0.778866,0.776969,0.774712,0.801526,0.805305,0.784057
4,10,10,10,50,0.091033,0.086893,0.108125,0.080405,0.105917,0.077083,...,0.786677,0.802271,0.806557,0.799614,0.789333,0.804087,0.787763,0.794948,0.819105,0.801781


In [13]:
np.shape(train)
train=train.sample(frac=1) #shuffle

In [14]:
#submission=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/데이터분석 /bus_ridetime_dacon/12회 대회/submission_제출양식.csv')
test=pd.read_csv('\data/test.csv')
test

,id,0,1,2,3,4,5,6,7,8,...,216,217,218,219,220,221,222,223,224,225
0,0,0.535410,0.520775,0.494087,0.465134,0.430339,0.401751,0.355986,0.326427,0.282340,...,0.748339,0.757575,0.768130,0.777062,0.769173,0.768253,0.738704,0.739460,0.702139,0.702238
1,1,0.351099,0.398179,0.413809,0.418529,0.433257,0.455410,0.451065,0.464230,0.476011,...,0.333931,0.276307,0.211513,0.159223,0.110982,0.083130,0.099780,0.145420,0.260501,0.343857
2,2,0.490537,0.435958,0.413428,0.355796,0.335777,0.299944,0.242745,0.210555,0.180739,...,0.709371,0.746826,0.781436,0.788292,0.828630,0.835166,0.845859,0.846032,0.836724,0.846779
3,3,0.051634,0.075802,0.133983,0.154546,0.209387,0.251700,0.287552,0.333000,0.340617,...,0.075046,0.056651,0.079884,0.147469,0.213112,0.298096,0.382823,0.489381,0.562383,0.599247
4,4,0.154031,0.201728,0.270414,0.283799,0.343050,0.340233,0.379244,0.378511,0.373017,...,0.255070,0.242396,0.271287,0.328828,0.397950,0.486436,0.530573,0.582752,0.637296,0.637238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.199957,0.227188,0.250628,0.265388,0.291736,0.319845,0.339820,0.368420,0.373319,...,0.422622,0.410483,0.409814,0.420162,0.426533,0.445706,0.487397,0.495991,0.534095,0.549278
9996,9996,0.180469,0.142472,0.112432,0.084084,0.071504,0.068979,0.070754,0.063943,0.071106,...,0.745875,0.776221,0.767607,0.774457,0.805273,0.802651,0.810866,0.792099,0.796827,0.791949
9997,9997,0.169476,0.180695,0.225148,0.220553,0.262136,0.288092,0.280675,0.312065,0.304840,...,0.282820,0.296270,0.324376,0.391588,0.436017,0.500170,0.569207,0.623997,0.673445,0.688012
9998,9998,0.218762,0.204023,0.207701,0.198991,0.188334,0.173722,0.161461,0.155859,0.136998,...,0.650535,0.662007,0.688480,0.708460,0.722464,0.726888,0.758949,0.771153,0.769234,0.785455


결측치 확인


In [16]:
train.isnull().sum()

layer_1    0
layer_2    0
layer_3    0
layer_4    0
0          0
          ..
221        0
222        0
223        0
224        0
225        0
Length: 230, dtype: int64

In [17]:
train.columns

Index(['layer_1', 'layer_2', 'layer_3', 'layer_4', '0', '1', '2', '3', '4',
       '5',
       ...
       '216', '217', '218', '219', '220', '221', '222', '223', '224', '225'],
      dtype='object', length=230)

In [18]:
# make neural network
#!pip install keras
from keras import*
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import metrics

In [19]:
from keras import backend as K
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score

In [20]:
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
learning_rate=0.01
import keras 
import tensorflow as tf
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import f1_score 
from keras.callbacks import ModelCheckpoint, EarlyStopping

def create_model():

    network=models.Sequential()
    network.add(layers.Dense(2000,activation='relu',input_shape=(226,)))
    network.add(BatchNormalization())  
    network.add(layers.Dense(4000, activation='relu'))
    network.add(Dropout(0.05))
    network.add(BatchNormalization())  
    network.add(layers.Dense(8000, activation='relu'))
    network.add(BatchNormalization())  
    network.add(Dropout(0.08))
    network.add(layers.Dense(7000, activation='relu'))
    network.add(BatchNormalization())  
    network.add(layers.Dense(3000, activation='relu'))
    network.add(Dropout(0.08))
    network.add(BatchNormalization())  
    network.add(layers.Dense(1000, activation='relu'))
    network.add(Dropout(0.08))
    network.add(BatchNormalization())  
    network.add(layers.Dense(300, activation='relu'))
    network.add(BatchNormalization())  
    network.add(layers.Dense(4,activation='relu'))

    opt=keras.optimizers.Adam(lr=learning_rate,decay=1e-6)
    #decay=업데이트 마다 적용되는 학습률의 감소값 

    earlystopping = EarlyStopping(monitor='val_loss',  # 모니터 기준 설정 (val loss) 
                              patience=10,         # 5회 Epoch동안 개선되지 않는다면 종료
                             )

    network.compile(optimizer=opt,loss='mean_absolute_error',metrics=[precision,recall,f1score])
    
    layer_4=train.iloc[:,0:4]
    layer_else=train.iloc[:,4:]
    history=network.fit(layer_else,layer_4, validation_split = 0.3,epochs=1000,batch_size=512,callbacks=[earlystopping])
    network.summary()
    return history

In [ ]:
history=create_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 567000 samples, validate on 243000 samples
Epoch 1/1000
567000/567000 [==============================] - 268s 472us/step - loss: 41.9394 - precision: 1.0000 - recall: 0.9449 - f1score: 0.9651 - val_loss: 20.3583 - val_precision: 1.0000 - val_recall: 0.9941 - val_f1score: 0.9970
Epoch 2/1000
567000/567000 [==============================] - 273s 481us/step - loss: 11.5523 - precision: 1.0000 - recall: 0.9969 - f1score: 0.9984 - val_loss: 9.7500 - val_precision: 1.0000 - val_recall: 0.9978 - val_f1score: 0.9989
Epoch 3/1000
567000/567000 [==============================] - 283s 499us/step - loss: 8.3122 - precision: 1.0000 - recall: 0.9989 - f1score: 0.9995 - val_loss: 9.3758 - val_precision: 1.0000 - val_recall: 0.9995 - val_f1score: 0.9997
Epoch 4/1000
567000/567000 [==============================] - 259s 457us/step - loss: 7.2064 - precision: 1.0000 - recall: 0.9993 - f1score: 0.9996 - val_loss: 7

567000/567000 [==============================] - 259s 456us/step - loss: 4.3009 - precision: 1.0000 - recall: 0.9995 - f1score: 0.9997 - val_loss: 2.3649 - val_precision: 1.0000 - val_recall: 1.0000 - val_f1score: 1.0000
Epoch 34/1000
567000/567000 [==============================] - 261s 461us/step - loss: 4.3480 - precision: 1.0000 - recall: 0.9994 - f1score: 0.9997 - val_loss: 3.3947 - val_precision: 1.0000 - val_recall: 0.9999 - val_f1score: 1.0000
Epoch 35/1000
567000/567000 [==============================] - 262s 462us/step - loss: 4.2619 - precision: 1.0000 - recall: 0.9995 - f1score: 0.9998 - val_loss: 2.4438 - val_precision: 1.0000 - val_recall: 0.9999 - val_f1score: 1.0000
Epoch 36/1000
567000/567000 [==============================] - 260s 459us/step - loss: 4.2612 - precision: 1.0000 - recall: 0.9995 - f1score: 0.9997 - val_loss: 3.6120 - val_precision: 1.0000 - val_recall: 1.0000 - val_f1score: 1.0000
Epoch 37/1000
260608/567000 [============>.................] - ETA: 2:11 -

In [ ]:
#history=network.fit(layer_else,layer_4, validation_split = 0.3,epochs=1000,batch_size=512,callbacks=[earlystopping])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.legend(['training_loss', 'validation_loss'], loc = 'upper right')
plt.show()

In [ ]:
print(min(history.history['val_loss']))

In [ ]:
plt.plot(history.history['recall'])
plt.plot(history.history['precision'])
plt.legend(['recall','precision'], loc = 'upper right')
plt.show()